# 제품 이상여부 판별 프로젝트


## 데이터 불러오기


### 필수 라이브러리


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier, VotingClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [2]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/train_data_0817.csv")
test_data = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/test_data_0817.csv")

In [3]:
test_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17361 entries, 0 to 17360
Data columns (total 39 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Set ID                                          17361 non-null  object 
 1   CURE SPEED Collect Result_Dam                   17361 non-null  int64  
 2   DISCHARGED SPEED OF RESIN Collect Result_Dam    17361 non-null  int64  
 3   Head Clean Position Z Collect Result_Dam        17361 non-null  float64
 4   Head Purge Position Z Collect Result_Dam        17361 non-null  float64
 5   Head Zero Position Y Collect Result_Dam         17361 non-null  float64
 6   Stage2_Circle_Distance_Speed_Dam                17361 non-null  float64
 7   WorkMode Collect Result                         17361 non-null  float64
 8   Chamber Temp. Collect Result_AutoClave          17361 non-null  int64  
 9   DISCHARGED SPEED OF RESIN Collect Resul

In [4]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [5]:
# 전체 공통 변수
### correlation 확인을 위한 변수 리스트
var_all_corr = [
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### train
var_all_train = [
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_receip_encoded',
    'workorder_receip_encoded'
]

In [6]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [7]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [8]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [9]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [10]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 38
train_data_dam DataFrame의 칼럼 수: 21
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 13
train_data_fill2 DataFrame의 칼럼 수: 13
----test data-----
test_data DataFrame의 칼럼 수: 39
test_data_dam DataFrame의 칼럼 수: 22
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 14
test_data_fill2 DataFrame의 칼럼 수: 14


---

## 모델링

### 모델 정의

In [11]:
# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model  # 학습된 모델 반환

# 사용 예시
# train_and_evaluate_model(
#     'lgbm', train_data_fill1,
#     n_estimators=979,
#     num_leaves=1565,
#     max_depth=34,
#     learning_rate=0.04888906225539191,
#     min_child_samples=36,
#     boosting_type='dart',
#     random_state=RANDOM_STATE,
#     verbose=-1
# )


def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

# 사용 예시
# model_dam = fit_all_train_data_function(
#     'lgbm', train_data_dam,
#     n_estimators=2748,
#     num_leaves=657,
#     max_depth=256,
#     learning_rate=0.001043279508273329,
#     min_child_samples=58,
#     boosting_type='dart',
#     random_state=RANDOM_STATE,
#     verbose=-1
# )

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

# # VotingClassifier 사용 예시
# estimators = [
#     ('dam', train_model_Dam),
#     ('autoclave', train_model_AutoClave),
#     ('fill1', train_model_Fill1),
#     ('fill2', train_model_Fill2)
# ]

# # VotingClassifier 학습 및 평가
# voting_clf_hard = train_and_evaluate_voting_classifier(train_data, estimators, voting='hard')
# voting_clf_soft = train_and_evaluate_voting_classifier(train_data, estimators, voting='soft', threshold=0.6)

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

# 사용 예시
# # 예측 확률 리스트 (소프트 보팅용)
# probs = [
#     model_Dam.predict_proba(x_test_dam)[:, 1],
#     model_AutoClave.predict_proba(x_test_autoclave)[:, 1],
#     model_Fill1.predict_proba(x_test_fill1)[:, 1],
#     model_Fill2.predict_proba(x_test_fill2)[:, 1]
# ]

# # 예측 리스트 (하드 보팅용)
# preds = [
#     model_Dam.predict(x_test_dam),
#     model_AutoClave.predict(x_test_autoclave),
#     model_Fill1.predict(x_test_fill1),
#     model_Fill2.predict(x_test_fill2)
# ]

# # 소프트 보팅 결과
# final_predictions_soft = voting(probs, method='soft', threshold=0.3)
# print(sum(final_predictions_soft))

# # 하드 보팅 결과
# final_predictions_hard = voting(preds, method='hard')
# print(sum(final_predictions_hard))



### 모델 학습

Dam 모델

In [12]:
train_model_Dam = train_and_evaluate_model(
    'cat', train_data_dam
    , iterations = 755
    , depth = 7
    , learning_rate = 0.19866742206648758
    , l2_leaf_reg = 0.24483628170369134
    , border_count = 95
    , random_state=RANDOM_STATE
)

0:	learn: 0.4580898	total: 165ms	remaining: 2m 4s
1:	learn: 0.3488357	total: 184ms	remaining: 1m 9s
2:	learn: 0.2881375	total: 203ms	remaining: 50.9s
3:	learn: 0.2502751	total: 220ms	remaining: 41.3s
4:	learn: 0.2282138	total: 238ms	remaining: 35.8s
5:	learn: 0.2163791	total: 257ms	remaining: 32.1s
6:	learn: 0.2088969	total: 275ms	remaining: 29.4s
7:	learn: 0.2031723	total: 302ms	remaining: 28.2s
8:	learn: 0.2001559	total: 324ms	remaining: 26.8s
9:	learn: 0.1977204	total: 349ms	remaining: 26s
10:	learn: 0.1959476	total: 371ms	remaining: 25.1s
11:	learn: 0.1946537	total: 397ms	remaining: 24.6s
12:	learn: 0.1935603	total: 418ms	remaining: 23.8s
13:	learn: 0.1929580	total: 441ms	remaining: 23.4s
14:	learn: 0.1921834	total: 461ms	remaining: 22.8s
15:	learn: 0.1916500	total: 481ms	remaining: 22.2s
16:	learn: 0.1909796	total: 501ms	remaining: 21.7s
17:	learn: 0.1902881	total: 523ms	remaining: 21.4s
18:	learn: 0.1898539	total: 546ms	remaining: 21.2s
19:	learn: 0.1894706	total: 571ms	remaining

AutoClave 모델

In [13]:
train_model_AutoClave = train_and_evaluate_model(
    'xgb', train_data_autoclave
    , n_estimators = 488
    , learning_rate = 0.27456156507923796
    , max_depth = 18
    , alpha = 0.001345329538356762
    , gamma = 0.001271261094255318
    , reg_alpha = 0.8757519133030134
    , reg_lambda = 0.08373579326505055
    , colsample_bytree = 0.8186279659279335
    , subsample = 0.24909941675865316
    , random_state=RANDOM_STATE
    , verbose=-1
)

c:\Users\KimDongyoung\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:46:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


xgb 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.1964285714285714
---
Confusion Matrix:
[[7193  469]
 [ 341   99]]
---
Accuracy: 0.900024685262898
Precision: 0.1742957746478873
Recall: 0.225




Fill1 모델

In [14]:
train_model_Fill1 = train_and_evaluate_model(
    'lgbm', train_data_fill1
    , n_estimators=1399
    , num_leaves=1456
    , max_depth=38
    , learning_rate=0.0020188481674757053 
    , min_child_samples=21
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_fill1 데이터로 학습한 결과:
F1 Score: 0.21714285714285714
---
Confusion Matrix:
[[7166  496]
 [ 326  114]]
---
Accuracy: 0.898543569489015
Precision: 0.18688524590163935
Recall: 0.2590909090909091




Fill2 모델

In [15]:
train_model_Fill2 = train_and_evaluate_model(
    'lgbm', train_data_fill2
    , n_estimators=1997
    , num_leaves=1641
    , max_depth=10
    , learning_rate=0.07435948250635764 
    , min_child_samples=114
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

lgbm 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.2166405023547881
---
Confusion Matrix:
[[7534  128]
 [ 371   69]]
---
Accuracy: 0.9384102690693656
Precision: 0.350253807106599
Recall: 0.15681818181818183




In [16]:
# VotingClassifier 사용 
estimators = [
    ('dam', train_model_Dam),
    ('autoclave', train_model_AutoClave),
    ('fill1', train_model_Fill1),
    ('fill2', train_model_Fill2)
]

# VotingClassifier 학습 및 평가
# voting_clf_hard = voting_function(train_data, estimators, voting='hard')
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.3)

0:	learn: 0.4828848	total: 26.3ms	remaining: 19.8s
1:	learn: 0.3663924	total: 50.9ms	remaining: 19.2s
2:	learn: 0.2965619	total: 69.4ms	remaining: 17.4s
3:	learn: 0.2545137	total: 91.3ms	remaining: 17.1s
4:	learn: 0.2333698	total: 116ms	remaining: 17.3s
5:	learn: 0.2219404	total: 139ms	remaining: 17.3s
6:	learn: 0.2132917	total: 159ms	remaining: 17s
7:	learn: 0.2067425	total: 184ms	remaining: 17.2s
8:	learn: 0.2015278	total: 204ms	remaining: 16.9s
9:	learn: 0.1989062	total: 225ms	remaining: 16.7s
10:	learn: 0.1972654	total: 250ms	remaining: 16.9s
11:	learn: 0.1960017	total: 271ms	remaining: 16.8s
12:	learn: 0.1944701	total: 292ms	remaining: 16.6s
13:	learn: 0.1939468	total: 313ms	remaining: 16.6s
14:	learn: 0.1926396	total: 334ms	remaining: 16.5s
15:	learn: 0.1917553	total: 354ms	remaining: 16.3s
16:	learn: 0.1912700	total: 377ms	remaining: 16.4s
17:	learn: 0.1905300	total: 399ms	remaining: 16.3s
18:	learn: 0.1901163	total: 424ms	remaining: 16.4s
19:	learn: 0.1897158	total: 445ms	remai

c:\Users\KimDongyoung\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:03:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Voting Classifier로 학습한 결과:
F1 Score: 0.2372413793103448
---
Confusion Matrix:
[[7463  199]
 [ 354   86]]
---
Accuracy: 0.9317452480868921
Precision: 0.3017543859649123
Recall: 0.19545454545454546




---

### 모델 학습(train 데이터 전체 학습)

위의 모델학습 코드에서 함수명만 바뀌고 들어가는 값들은 동일  
-> 위의 코드 복붙한다음 함수명만 바꿔주면 사용하기 편함  

In [17]:
model_Dam = fit_all_train_data_function(
    'cat', train_data_dam
    , iterations = 755
    , depth = 7
    , learning_rate = 0.19866742206648758
    , l2_leaf_reg = 0.24483628170369134
    , border_count = 95
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'xgb', train_data_autoclave
    , n_estimators = 488
    , learning_rate = 0.27456156507923796
    , max_depth = 18
    , alpha = 0.001345329538356762
    , gamma = 0.001271261094255318
    , reg_alpha = 0.8757519133030134
    , reg_lambda = 0.08373579326505055
    , colsample_bytree = 0.8186279659279335
    , subsample = 0.24909941675865316
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill1 = fit_all_train_data_function(
    'lgbm', train_data_fill1
    , n_estimators=1399
    , num_leaves=1456
    , max_depth=38
    , learning_rate=0.0020188481674757053 
    , min_child_samples=21
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

model_Fill2 = fit_all_train_data_function(
    'lgbm', train_data_fill2
    , n_estimators=1997
    , num_leaves=1641
    , max_depth=10
    , learning_rate=0.07435948250635764 
    , min_child_samples=114
    , boosting_type='dart'
    , random_state=RANDOM_STATE
    , verbose=-1
)

0:	learn: 0.4578706	total: 26.8ms	remaining: 20.2s
1:	learn: 0.3474299	total: 48.4ms	remaining: 18.2s
2:	learn: 0.2869435	total: 69.8ms	remaining: 17.5s
3:	learn: 0.2492813	total: 96.3ms	remaining: 18.1s
4:	learn: 0.2262987	total: 123ms	remaining: 18.4s
5:	learn: 0.2146183	total: 141ms	remaining: 17.6s
6:	learn: 0.2072925	total: 164ms	remaining: 17.5s
7:	learn: 0.2015633	total: 183ms	remaining: 17.1s
8:	learn: 0.1983093	total: 203ms	remaining: 16.9s
9:	learn: 0.1957367	total: 223ms	remaining: 16.6s
10:	learn: 0.1939494	total: 246ms	remaining: 16.6s
11:	learn: 0.1928495	total: 266ms	remaining: 16.5s
12:	learn: 0.1921533	total: 288ms	remaining: 16.4s
13:	learn: 0.1914628	total: 312ms	remaining: 16.5s
14:	learn: 0.1910411	total: 334ms	remaining: 16.5s
15:	learn: 0.1902580	total: 355ms	remaining: 16.4s
16:	learn: 0.1895844	total: 376ms	remaining: 16.3s
17:	learn: 0.1892807	total: 397ms	remaining: 16.3s
18:	learn: 0.1888425	total: 416ms	remaining: 16.1s
19:	learn: 0.1885503	total: 436ms	rem

c:\Users\KimDongyoung\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:25:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


xgb 모델이 train_data_autoclave 데이터로 학습 완료
lgbm 모델이 train_data_fill1 데이터로 학습 완료
lgbm 모델이 train_data_fill2 데이터로 학습 완료


---

In [18]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)


Voting(hard)

4개의 모델중 다수의 모델이 판단한 값을 타겟값으로 가져감

In [19]:
# 예측 리스트 (하드 보팅용)
preds = [
    model_Dam.predict(x_test_dam),
    model_AutoClave.predict(x_test_autoclave),
    model_Fill1.predict(x_test_fill1),
    model_Fill2.predict(x_test_fill2)
]

# 하드 보팅 결과
final_predictions = voting(preds, method='hard')
print(sum(final_predictions))

73


Voting(soft)

4개의 모델의 확률의 합에 대한 평균이 일정 수치(스레스홀드) 값을 넘으면 AbNormal 값을 가짐

In [20]:
# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1],
    model_AutoClave.predict_proba(x_test_autoclave)[:, 1],
    model_Fill1.predict_proba(x_test_fill1)[:, 1],
    model_Fill2.predict_proba(x_test_fill2)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.3)
print(sum(final_predictions))

440


위의 코드와 아래의 코드 동일 내용

In [21]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.26 적용
final_predictions = (soft_voting_probs >= 0.26).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

680


위의 결과값 ~ Public Score : 

In [22]:
# 소프트 보팅: 각 모델의 확률 평균 계산
soft_voting_probs = np.mean(probs, axis=0)

# 최종 예측: 평균 확률에 대해 스레드 홀드 0.26 적용
final_predictions = (soft_voting_probs >= 0.25).astype(int)

# 최종 예측 결과 출력
print(sum(final_predictions))

746


## 4. 제출하기


### 제출 파일 작성


In [28]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("submission_model_train3.csv", index=False)

In [29]:
df_sub['target'].value_counts()

target
Normal      16615
AbNormal      746
Name: count, dtype: int64

In [25]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,AbNormal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,AbNormal
9,002d904240d84b188d410d16383a9c3a,Normal


**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**


---

## 5. ETC...

모델을 통해 예측해 낸 정답지(csv) 파일의 결과값을 불러와 voting 하는 방식  
이때의 voting은 hard방식만 가능(예측확률은 저장되어있지 않음으로 soft 보팅은 불가능)  

In [26]:
import pandas as pd
import numpy as np

def read_submission_files(file_paths):
    """
    제출 파일을 읽어와서 예측 결과를 반환합니다.

    Parameters:
    file_paths (list of str): 제출 파일 경로 리스트

    Returns:
    list of np.array: 각 제출 파일의 예측 결과 리스트
    """
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    """
    하드 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds (list of np.array): 각 모델의 예측 배열 리스트

    Returns:
    np.array: 최종 예측 결과
    """
    preds = np.array(preds)
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

In [ ]:
# 제출 파일 경로 리스트
file_paths = [
    "submission1.csv"
    , "submission2.csv"
    , "submission3.csv"
    , "submission4.csv"
    # 파일 추가 가능
    ]

# 제출 파일에서 예측 결과 읽어오기
predictions = read_submission_files(file_paths)

# 하드 보팅 결과
final_predictions_hard = hard_voting(predictions)

# 결과를 새로운 제출 파일로 저장
df_sub = pd.read_csv(file_paths[0])
df_sub["target"] = final_predictions_hard
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
df_sub.to_csv("final_submission.csv", index=False)

print("최종 제출 파일이 'final_submission.csv'로 저장되었습니다.")

In [ ]:
df_sub['target'].value_counts()

Normal      17234
AbNormal      127
Name: target, dtype: int64

In [ ]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,Normal
9,002d904240d84b188d410d16383a9c3a,Normal
